<a href="https://colab.research.google.com/github/ricardodeazambuja/elmer_fem_playground/blob/main/Testing_Elmer_FEM_gmsh_pyelmer_meshio_and_plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Elmer
!sudo apt-add-repository ppa:elmer-csc-ubuntu/elmer-csc-ppa -y > /dev/null
!sudo DEBIAN_FRONTEND=noninteractive apt-get update -qq < /dev/null > /dev/null
!sudo DEBIAN_FRONTEND=noninteractive apt-get install elmerfem-csc -qq < /dev/null > /dev/null

# gmsh
!sudo DEBIAN_FRONTEND=noninteractive apt-get install gmsh -qq < /dev/null > /dev/null
!pip -q install --upgrade gmsh > /dev/null

# 
!pip -q install --upgrade meshio > /dev/null

# pyelmer
!pip -q install --upgrade pyelmer > /dev/null
!pip -q install --upgrade objectgmsh > /dev/null

In [2]:
import os
import numpy as np

import gmsh
from pyelmer import elmer
from pyelmer import execute
from pyelmer.post import scan_logfile
# from objectgmsh import add_physical_group, get_boundaries_in_box

import meshio

In [3]:
run_mesher = True
run_gmsh_gui = False # the gui will NOT work on google colab!
simdir = "simdata"

path = os.path.join(os.getcwd(), simdir)
if not os.path.isdir(path):
  os.mkdir(path)

map_size = 100,100,100
heat_source_pos = 80.0,80.0,80.0
obs_meas = 40, 40, 50, 30, 30, 100

In [4]:
gmsh.initialize()

gmsh.model.add("3d_heat_experiment")
geom = gmsh.model.occ

In [5]:
# map box
m1 = geom.addBox(0, 0, 0, map_size[0], map_size[1], map_size[2], tag=1)
m1

1

In [6]:
# obstacle
obs1 = geom.addBox(obs_meas[0], obs_meas[1], obs_meas[2], 
                   obs_meas[3], obs_meas[4], obs_meas[5], tag=2)
obs1

2

In [7]:
# remove volumes (removeObject=True) and remove the tools used to cut (removeTool=True)
cut_result = geom.cut([(3, m1)], [(3, obs1)], removeObject=True, removeTool=True)
print(cut_result)
geom.synchronize()

([(3, 1)], [[(3, 1)], []])


In [8]:
# Get all the 2D elements (dim=2)
surfaces = gmsh.model.occ.getEntities(dim=2)
surfaces

[(2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11)]

In [9]:
# Get all the 2D elements (dim=2) inside the bounding box
obs_surfaces = geom.getEntitiesInBoundingBox(
                              obs_meas[0]-0.1,
                              obs_meas[1]-0.1,
                              obs_meas[2]-0.1,
                              obs_meas[0]+obs_meas[3]+0.1,
                              obs_meas[1]+obs_meas[3]+0.1,
                              obs_meas[2]+obs_meas[5]+0.1,
                              dim=2)
obs_surfaces

[(2, 7), (2, 8), (2, 9), (2, 10), (2, 11)]

In [10]:
# To get the boundaries (surfaces) of the map, we get all and
# compare them to the ones from the obstacles
map_surfaces = []
for surface in surfaces:
    if surface not in obs_surfaces:
        map_surfaces.append(surface)
map_surfaces

[(2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6)]

In [11]:
# Elmer will only "see" physical groups, so we need to create them.
# Physical groups are separated by their dimension and the indices cannot repeat in the same dimension.
# Here it's dim=2 because map_surfaces[0][0] is 2 as it's a surface

# map_boundaries is the index of the 2D physical group that has all the map surfaces
map_boundaries = 1
ph_map_bound = gmsh.model.addPhysicalGroup(map_surfaces[0][0], [s[1] for s in map_surfaces], map_boundaries)
gmsh.model.setPhysicalName(map_surfaces[0][0], ph_map_bound, "Map_boundaries")
geom.synchronize()

In [12]:
# obs_boundaries is the index of the 2D physical group that has all the obstacle surfaces
obs_boundaries = 2
ph_obs_bound = gmsh.model.addPhysicalGroup(obs_surfaces[0][0], [s[1] for s in obs_surfaces], obs_boundaries)
gmsh.model.setPhysicalName(obs_surfaces[0][0], ph_obs_bound, "Obs_boundaries")
geom.synchronize()

In [13]:
# This will give us a list of all volumes (dim=3) defined
volumes = gmsh.model.getEntities(dim=3)
volumes

[(3, 1)]

In [14]:
# Finally, we will create a physical group for the volumes
map_marker = 1
ph_map_vol = gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], map_marker)
gmsh.model.setPhysicalName(volumes[0][0], ph_map_vol, "Map_volume")
geom.synchronize()

In [15]:
# Now we check the 2D entities indices for the physical group:
# - map_boundaries
print(gmsh.model.get_entities_for_physical_group(2,map_boundaries))
# - obs_boundaries
print(gmsh.model.get_entities_for_physical_group(2,obs_boundaries))
# And the 3D entities indices for the physical group map_marker
print(gmsh.model.get_entities_for_physical_group(3,map_marker))

[1 2 3 4 5 6]
[ 7  8  9 10 11]
[1]


I think I modified this from pyelmer examples...

In [16]:
############################################################################
### Meshing
############################################################################

# We can activate the calculation of mesh element sizes based on curvature
# (here with a target of 90 elements per 2*Pi radians):
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 90)

# Finally we apply an elliptic smoother to the grid to have a more regular
# mesh:
gmsh.option.setNumber("Mesh.Smoothing", 10)
gmsh.option.setNumber("Mesh.Algorithm3D", 10)  # faster
gmsh.option.setNumber('General.NumThreads', 12)
# gmsh.option.setNumber("Mesh.MeshSizeMin", 0.1)
# gmsh.option.setNumber("Mesh.MeshSizeMax", 1.0)

if run_mesher:
    geom.synchronize()
    gmsh.model.mesh.generate(dim=3)
    gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 0)
    gmsh.model.mesh.refine()
    gmsh.write(simdir+"/3d_box_with_whole.msh")
    gmsh.write(simdir+"/3d_box_with_whole.stl")
    #gmsh.write(simdir+"/3d_box_with_whole.geo_unrolled") #gmsh format

# Preview mesh (it WON'T work on colab...)
if run_gmsh_gui:
    gmsh.fltk.run()

# Clear mesh and close gmsh API.
gmsh.clear()
gmsh.finalize()

In [17]:
# %%time
execute.run_elmer_grid(simdir, "3d_box_with_whole.msh")

In [18]:
# %%time

###############
# elmer setup
sim = elmer.Simulation()
sim.settings = {"Max Output Level":"5",
                "Coordinate System": "Cartesian", 
                "Coordinate Mapping(3)": "1 2 3",
                "Simulation Type": "Steady state",
                "Steady State Max Iterations": "1",
                "Output Intervals(1)": "1",
                "Solver Input File": "case.sif",
                "Post File" : "case.vtu"}

ideal = elmer.Material(sim, "ideal")
ideal.data = {"Heat Conductivity": 10.0, 
              "Density": 1.0,
              "Heat Capacity": 1.0}

solver_heat = elmer.Solver(sim, "heat_solver")
solver_heat.data = {
    "Equation": "HeatSolver",
    "Procedure": '"HeatSolve" "HeatSolver"',
    "Variable": '"Temperature"',
    "Exec Solver": "Always",
    "Stabilize": "True",
    "Optimize Bandwidth": "True",
    "Steady State Convergence Tolerance": 1.0e-5,
    "Nonlinear System Convergence Tolerance": 1.0e-7,
    "Nonlinear System Max Iterations": 20,
    "Nonlinear System Newton After Iterations": 3,
    "Nonlinear System Newton After Tolerance": 1.0e-3,
    "Nonlinear System Relaxation Factor": 1,
    "Linear System Solver": "Iterative",
    "Linear System Iterative Method": "BiCGStab",
    "Linear System Max Iterations": 500,
    "Linear System Convergence Tolerance": 1.0e-10,
    "BiCGstabl polynomial degree": 2,
    "Linear System Preconditioning": "ILU0",
    "Linear System ILUT Tolerance": 1.0e-3,
    "Linear System Abort Not Converged": "False",
    "Linear System Residual Output": 10,
    "Linear System Precondition Recompute": 1
}
  
eqn = elmer.Equation(sim, "Heat Equation", [solver_heat])

T0 = elmer.InitialCondition(sim, "T0", {"Temperature": 0})


bdy_ideal = elmer.Body(sim, "main_body", [map_marker])
bdy_ideal.material = ideal
bdy_ideal.initial_condition = T0
bdy_ideal.equation = eqn

#
# Using body force to inject "heat"
#

# Using an IF to turn heat on only in ONE point, something problematic for FEA stuff
# https://www.elmerfem.org/forum/viewtopic.php?p=27108
# expr = f"Variable Coordinate; Real MATC \"\
# if (sqrt((tx(0)-({heat_source_pos[0]}))^2+(tx(1)-({heat_source_pos[1]}))^2+(tx(2)-({heat_source_pos[2]}))^2)<13) \
# (1); else (0);\""

# Using an equation to distribute heat that falls with distance
# expr = f"Variable Coordinate; Real MATC \"\
# 1000/sqrt((tx(0)-({heat_source_pos[0]}))^2+(tx(1)-({heat_source_pos[1]}))^2+(tx(2)-({heat_source_pos[2]}))^2);\""
# force_goal = elmer.BodyForce(sim, "heat_source", 
#                               {"Heat Source": expr})

# Now we need to add this body force to the body where we want to apply it
# bdy_ideal.body_force = force_goal



bound_obs = elmer.Boundary(sim, "obstacles", [obs_boundaries], {"Temperature": 0})
bound_obs = elmer.Boundary(sim, "rest", [map_boundaries], {"Temperature": 0})

# Using a boundary is MUCH MUCH faster than using MATC, 1/4 of the time!
bound_obs = elmer.Boundary(sim, "rest2", None, 
                           {"Target Coordinates(1,3)":f"Real {heat_source_pos[0]} {heat_source_pos[1]} {heat_source_pos[2]}",
                            "Temperature Load": "Real 1000"})


sim.write_startinfo(simdir)
sim.write_sif(simdir)

Wrote sif-file.


In [19]:
# %%time

##############
# execute ElmerGrid & ElmerSolver
execute.run_elmer_solver(simdir)

###############
# scan log for errors and warnings
err, warn, stats = scan_logfile(simdir)
print("Errors:", err)
print("Warnings:", warn)
print("Statistics:", stats)

Errors: []
Warnings: []
Statistics: {'CPU-time': 0.25, 'real-time': 0.43}


In [20]:
#
# Meshio is the easiest way to load the results...
#
# https://computational-acoustics.gitlab.io/website/posts/20-intro-to-meshio/
data = meshio.read(simdir+"/case_t0001.vtu")

In [21]:
# https://chart-studio.plotly.com/~empet/14749.embed
import plotly.graph_objects as go

In [22]:
vertices = data.points
triangles =  data.cells_dict['triangle']
intensities = data.point_data['temperature'][:,0]

In [25]:
x,y,z = vertices.T
I,J,K = triangles.T
tri_points = vertices[triangles]

In [26]:
pl_mesh = go.Mesh3d(x=x,
                    y=y,
                    z=z,
                    intensity = intensities,
                    flatshading=True,
                    i=I,
                    j=J,
                    k=K,
                    showscale=True
                    )

In [27]:
# triangles

Xe = []
Ye = []
Ze = []
for T in tri_points:
    Xe.extend([T[k%3][0] for k in range(4)]+[ None])
    Ye.extend([T[k%3][1] for k in range(4)]+[ None])
    Ze.extend([T[k%3][2] for k in range(4)]+[ None])
       
#define the trace for triangle sides
lines = go.Scatter3d(
                   x=Xe,
                   y=Ye,
                   z=Ze,
                   mode='lines',
                   name='',
                   line=dict(color='black', width=5))  

In [28]:
layout = go.Layout(
         title="Results from simulation<br>Mesh3d with flatshading",
         font=dict(size=16, color='white'),
         width=700,
         height=700,
         scene_xaxis_visible=False,
         scene_yaxis_visible=False,
         scene_zaxis_visible=False,
         paper_bgcolor='rgb(50,50,50)',
        )

In [29]:
#fig = go.Figure(data=[pl_mesh], layout=layout)
#fig = go.Figure(data=[lines], layout=layout)
#fig = go.Figure(data=[pl_mesh, lines])
fig = go.Figure(data=[pl_mesh, lines], layout=layout)
fig.show()

In [33]:
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=5,
        color=np.log(intensities+1),    # set color to an array/list of desired values
        colorscale='Viridis',           # choose a colorscale
        showscale=True,
        opacity=0.8,
        #cmax=0.6,
        #cmin=0.4
    ),
)])

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()